In [210]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [211]:
instrument_route = '/Users/nathanjones/Downloads/NUWE/Hackathons/Schneider_DataScience/hackathon-schneider-pollution/data/raw/instrument_data.csv'

df = pd.read_csv(instrument_route, parse_dates=["Measurement date"])

In [212]:
station_ids = [205, 209, 223, 224, 226, 227]

In [213]:
df['Measurement date'] = df['Measurement date'].dt.floor('h')

In [214]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3703662 entries, 0 to 3703661
Data columns (total 5 columns):
 #   Column             Dtype         
---  ------             -----         
 0   Measurement date   datetime64[ns]
 1   Station code       int64         
 2   Item code          int64         
 3   Average value      float64       
 4   Instrument status  int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 141.3 MB


<mark>feature engineering</mark>

In [215]:
def feature_engineering(df, is_prediction=False):

    items = {0: "SO2", 2: "NO2", 4: "CO", 5: "O3", 7: "PM10", 8: "PM2.5"}
    # Pollutant column
    df['Pollutant'] = df['Item code'].map(items)
    
    # Time-based features
    df["Hour_sin"] = np.sin(2 * np.pi * df["Measurement date"].dt.hour / 24)
    df["Hour_cos"] = np.cos(2 * np.pi * df["Measurement date"].dt.hour / 24)

    df["Day_sin"] = np.sin(2 * np.pi * df["Measurement date"].dt.dayofweek / 7)
    df["Day_cos"] = np.cos(2 * np.pi * df["Measurement date"].dt.dayofweek / 7)

    df["Month_sin"] = np.sin(2 * np.pi * df["Measurement date"].dt.month / 12)
    df["Month_cos"] = np.cos(2 * np.pi * df["Measurement date"].dt.month / 12)

    # Sort the dataframe by date to calculate rolling average
    df = df.sort_values(by=["Station code", "Item code", "Measurement date"])

    # Set 'Measurement date' as the index temporarily
    df.set_index("Measurement date", inplace=True)

    # Apply rolling averages within each 'Station code' and 'Item code' group (6-month rolling window)
    df["6_Month_Avg"] = (
        df.groupby(["Station code", "Item code"])["Average value"]
        .rolling(window="180d", min_periods=1)  # 180 days = 6 months
        .mean().round(3)
        .reset_index(level=[0, 1], drop=True)
    )

    # Reset the index to restore 'Measurement date' as a column
    df.reset_index(inplace=True)

    # If this is for prediction, we should remove rows where the 6-month average is not available
    if is_prediction:
        df = df[df["6_Month_Avg"].notna()]

    return df



In [216]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

# Copy entire DataFrame to handle Instrument status (binary)
binary_df = df.copy()  
binary_df['Instrument status'] = np.where(binary_df['Instrument status'] == 0, 0, 1)

# Define features and target
X = binary_df.drop(columns=['Instrument status'])  # Keep Measurement date and Pollutant for feature engineering
y = binary_df['Instrument status']

# Split data into train and test first to avoid data leakage
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Now apply feature engineering separately for training and testing sets
# Apply to training data
train_data = pd.concat([X_train, y_train], axis=1)  # Concatenate to keep the target
train_data = feature_engineering(train_data, is_prediction=False)

# Apply to testing data
test_data = pd.concat([X_test, y_test], axis=1)  # Concatenate to keep the target
test_data = feature_engineering(test_data, is_prediction=True)

# Separate features and target again after feature engineering
X_train = train_data.drop(columns=['Instrument status', 'Measurement date', 'Pollutant'])
y_train = train_data['Instrument status']
X_test = test_data.drop(columns=['Instrument status', 'Measurement date', 'Pollutant'])
y_test = test_data['Instrument status']

# Handle imbalance using class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

# Train Random Forest Model
rf_model = RandomForestClassifier(n_estimators=50, 
                                  class_weight=class_weight_dict,
                                  max_depth=10, 
                                  min_samples_split=5,
                                  random_state=42)
rf_model.fit(X_train, y_train)

# Predictions
y_pred = rf_model.predict(X_test)

# Evaluate Model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[704360  16980]
 [  6872  12521]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.98    721340
           1       0.42      0.65      0.51     19393

    accuracy                           0.97    740733
   macro avg       0.71      0.81      0.75    740733
weighted avg       0.98      0.97      0.97    740733



In [218]:
# Filter out only faulty cases (Instrument status != 0)
faulty_df = df[df['Instrument status'] != 0].copy()

# Define features and target for faulty data
X_faulty = faulty_df.drop(columns=['Instrument status'])  # Keep Measurement date and Pollutant for feature engineering
y_faulty = faulty_df['Instrument status']  # Multi-class target

print(y_faulty.value_counts())

# Split data into train and test first to avoid data leakage
X_train_faulty, X_test_faulty, y_train_faulty, y_test_faulty = train_test_split(
    X_faulty, y_faulty, test_size=0.2, stratify=y_faulty, random_state=42
)

# Now apply feature engineering separately for training and testing sets
# Apply to training data
train_faulty_data = pd.concat([X_train_faulty, y_train_faulty], axis=1)  # Concatenate to keep the target
train_faulty_data = feature_engineering(train_faulty_data, is_prediction=False)

# Apply to testing data
test_faulty_data = pd.concat([X_test_faulty, y_test_faulty], axis=1)  # Concatenate to keep the target
test_faulty_data = feature_engineering(test_faulty_data, is_prediction=True)

# Separate features and target again after feature engineering
X_train_faulty = train_faulty_data.drop(columns=['Instrument status', 'Measurement date', 'Pollutant'])
y_train_faulty = train_faulty_data['Instrument status']
X_test_faulty = test_faulty_data.drop(columns=['Instrument status', 'Measurement date', 'Pollutant'])
y_test_faulty = test_faulty_data['Instrument status']



# Train Multi-Class Random Forest Model
rf_multi = RandomForestClassifier(n_estimators=30, 
                                  max_depth=10, 
                                  min_samples_split=5,
                                  random_state=42)
rf_multi.fit(X_train_faulty, y_train_faulty)

# Predictions
y_pred_faulty = rf_multi.predict(X_test_faulty)

# Evaluate Model
print(confusion_matrix(y_test_faulty, y_pred_faulty))
print(classification_report(y_test_faulty, y_pred_faulty))

Instrument status
8    28323
1    27147
9    19668
4    17960
2     3868
Name: count, dtype: int64
[[4714    2   53  477  183]
 [ 170  214   27  142  221]
 [ 100   12 3358   96   26]
 [ 564   21   19 4878  183]
 [ 653   17   49  134 3081]]
              precision    recall  f1-score   support

           1       0.76      0.87      0.81      5429
           2       0.80      0.28      0.41       774
           4       0.96      0.93      0.95      3592
           8       0.85      0.86      0.86      5665
           9       0.83      0.78      0.81      3934

    accuracy                           0.84     19394
   macro avg       0.84      0.74      0.77     19394
weighted avg       0.84      0.84      0.83     19394



In [219]:
import joblib

joblib.dump(rf_model, f"/Users/nathanjones/Downloads/NUWE/Hackathons/Schneider_DataScience/hackathon-schneider-pollution/models/task_3/rf_model_task_3.pkl")
joblib.dump(rf_multi, f"/Users/nathanjones/Downloads/NUWE/Hackathons/Schneider_DataScience/hackathon-schneider-pollution/models/task_3/rf_multi_task_3.pkl")



['/Users/nathanjones/Downloads/NUWE/Hackathons/Schneider_DataScience/hackathon-schneider-pollution/models/task_3/rf_multi_task_3.pkl']

<mark>new dataframe for prediction</mark>

In [351]:
# Define the periods and station codes
query_dict = {
    205: {'Pollutant': 'SO2', 'start': '2023-11-01 00:00:00', 'end': '2023-11-30 23:00:00'},
    209: {'Pollutant': 'NO2', 'start': '2023-09-01 00:00:00', 'end': '2023-09-30 23:00:00'},
    223: {'Pollutant': 'O3', 'start': '2023-07-01 00:00:00', 'end': '2023-07-31 23:00:00'},
    224: {'Pollutant': 'CO', 'start': '2023-10-01 00:00:00', 'end': '2023-10-31 23:00:00'},
    226: {'Pollutant': 'PM10', 'start': '2023-08-01 00:00:00', 'end': '2023-08-31 23:00:00'},
    227: {'Pollutant': 'PM2.5', 'start': '2023-12-01 00:00:00', 'end': '2023-12-31 23:00:00'}
}



In [362]:
# Step 1: Create a DataFrame from the query_dict for the given periods and station codes
new_data = []

for station_code, query in query_dict.items():
    # Generate a date range for the period
    date_range = pd.date_range(start=query['start'], end=query['end'], freq='h')
    
    # Generate rows of data for each station code and pollutant
    for date in date_range:
        new_data.append({
            'Station code': station_code,
            'Pollutant': query['Pollutant'],
            'Measurement date': date,
            'Average value': np.nan  # Placeholder for the actual measurements
        })

# Convert new_data to a DataFrame
new_df = pd.DataFrame(new_data)

# Step 2: Concatenate the new data with your original training data (df)
combined_df = pd.concat([df, new_df], ignore_index=True)
combined_df_1 = pd.concat([df, new_df], ignore_index=True)

In [353]:
def feature_engineering_pred(df, is_prediction=False):
    items = {0: "SO2", 2: "NO2", 4: "CO", 5: "O3", 7: "PM10", 8: "PM2.5"}

    # Map the Item code to Pollutant
    df['Pollutant'] = df['Item code'].map(items)

    # Extract time-based features (sin/cos for cyclic features like hour, day, month)
    df["Hour_sin"] = np.sin(2 * np.pi * df["Measurement date"].dt.hour / 24)
    df["Hour_cos"] = np.cos(2 * np.pi * df["Measurement date"].dt.hour / 24)
    df["Day_sin"] = np.sin(2 * np.pi * df["Measurement date"].dt.dayofweek / 7)
    df["Day_cos"] = np.cos(2 * np.pi * df["Measurement date"].dt.dayofweek / 7)
    df["Month_sin"] = np.sin(2 * np.pi * df["Measurement date"].dt.month / 12)
    df["Month_cos"] = np.cos(2 * np.pi * df["Measurement date"].dt.month / 12)

    # Ensure 'Measurement date' is in datetime format
    df['Measurement date'] = pd.to_datetime(df['Measurement date'])

    # Drop duplicate entries based on key columns for prediction periods
    df = df.drop_duplicates(subset=["Measurement date", "Station code", "Item code"])

    # Create a composite index to ensure uniqueness (Measurement date + Station code + Item code)
    df.set_index(["Measurement date", "Station code", "Item code"], inplace=True)

    # Apply rolling averages (6 months for historical data, min_periods=1 for prediction)
    if not is_prediction:
        df = df.sort_values(by=["Station code", "Item code", "Measurement date"])

        # 6-month (180 days) rolling average
        df["6_Month_Avg"] = (
            df.groupby(["Station code", "Item code"])["Average value"]
            .rolling(window="180d", min_periods=1)  # 180 days = 6 months
            .mean()
            .reset_index(level=[0, 1], drop=True)
        )

    if is_prediction:
        # For prediction, apply rolling averages even if no past data exists.
        # Ensure we use a proper numeric window
        df["6_Month_Avg"] = (
            df.groupby(["Station code", "Item code"])["Average value"]
            .rolling(window=180, min_periods=1)  # Use numeric window size (in hours or data points)
            .mean()
            .reset_index(level=[0, 1], drop=True)
        )

    # Reset the index to restore 'Measurement date' as a column
    df.reset_index(inplace=True)

    return df


In [354]:
# Step 3: Apply your feature engineering to the combined DataFrame
# Assuming feature_engineering is the function you defined before
combined_df = feature_engineering_pred(combined_df, is_prediction=True)

In [355]:
# Step 4: Prepare the features for prediction (drop unnecessary columns)
X_combined = combined_df[['Station code', 'Item code', 'Average value', 'Hour_sin',
       'Hour_cos', 'Day_sin', 'Day_cos', 'Month_sin', 'Month_cos',
       '6_Month_Avg']]

In [356]:
# Step 5: Make binary predictions on the new data
predictions = rf_multi.predict(X_combined)

In [363]:
# Step 6: Add binary predictions to the DataFrame
combined_df_1['Predicted Instrument status'] = predictions

In [ ]:
#check results exist in given period
combined_df_1.loc[combined_df["Station code"]==205]

,Measurement date,Station code,Item code,Average value,Instrument status,Pollutant,Predicted Instrument status
155430,2021-01-01 00:00:00,205,0.0,0.006,0.0,NaN,9
155431,2021-01-01 00:00:00,205,2.0,0.068,0.0,NaN,9
155432,2021-01-01 00:00:00,205,4.0,1.300,0.0,NaN,9
155433,2021-01-01 00:00:00,205,5.0,0.002,0.0,NaN,9
155434,2021-01-01 00:00:00,205,7.0,77.000,0.0,NaN,9
...,...,...,...,...,...,...,...
3704377,2023-11-30 19:00:00,205,NaN,NaN,NaN,SO2,8
3704378,2023-11-30 20:00:00,205,NaN,NaN,NaN,SO2,8
3704379,2023-11-30 21:00:00,205,NaN,NaN,NaN,SO2,8
3704380,2023-11-30 22:00:00,205,NaN,NaN,NaN,SO2,8


In [365]:
# step 7 
# To store results in JSON format
target = {}

# Loop through the query_dict and filter the dataframe accordingly
for station_code, query in query_dict.items():
    # Filter the dataframe by Station code, Pollutant, and Date range
    filtered_df = combined_df_1[
        (combined_df_1['Station code'] == station_code) &
        (combined_df_1['Pollutant'] == query['Pollutant']) &
        (combined_df_1['Measurement date'] >= query['start']) &
        (combined_df_1['Measurement date'] <= query['end'])
    ]
    
    # Select only the columns needed for predictions
    prediction_columns = ['Measurement date', 'Station code', 'Pollutant', 'Predicted Instrument status']
    
    filtered_df = filtered_df[prediction_columns]
    
    # Convert to dictionary format for JSON
    target[station_code] = filtered_df.set_index('Measurement date').to_dict(orient='index')


In [ ]:
import json

# Convert to the required JSON format
formatted_target = {
    "target": {
        station_code: {
            timestamp.strftime("%Y-%m-%d %H:%M:%S"): data["Predicted Instrument status"]
            for timestamp, data in records.items()
        }
        for station_code, records in target.items()
    }
}

# Convert to JSON string
json_output = json.dumps(formatted_target, indent=4)

#with open("/Users/nathanjones/Downloads/NUWE/Hackathons/Schneider_DataScience/hackathon-schneider-pollution/predictions/predictions_task_3.json", "w") as f:
   # json.dump(formatted_target, f, indent=2)
